In [36]:
import pandas as pd
import numpy as np

In [83]:
# 이자보상비율에서 사용
kgaap = pd.read_csv('데이터/data/케이갭71.csv', encoding='CP949')
ifrscon = pd.read_csv('데이터/1차 전처리/연결별도_72피처병합.csv', encoding='CP949')

# 매출액증가율에서 사용
kgconsep = pd.read_csv('데이터/1차 전처리/연결별도_gaap성장성병합.csv', encoding='CP949')
consep = pd.read_csv('데이터/1차 전처리/연결별도_ifrs성장성병합.csv', encoding='CP949')


# 연결, 병합 데이터 비교하고 확인
print('<케이갭71>')
print('행렬 수 : ', kgaap.shape)
print('회사 수 : ', kgaap['회사명'].nunique())

print('\n<연결별도_72피처병합>')
print('행렬 수 : ', ifrscon.shape)
print('회사 수 : ', ifrscon['회사명'].nunique())


print('\n\n<연결별도_gaap성장성병합>')
print('행렬 수 : ', kgconsep.shape)
print('회사 수 : ', kgconsep['회사명'].nunique())

print('\n<연결별도_ifrs성장성병합>')
print('행렬 수 : ', consep.shape)
print('회사 수 : ', consep['회사명'].nunique())

<케이갭71>
행렬 수 :  (20831, 74)
회사 수 :  2388

<연결별도_72피처병합>
행렬 수 :  (18985, 75)
회사 수 :  2371


<연결별도_gaap성장성병합>
행렬 수 :  (20831, 10)
회사 수 :  2388

<연결별도_ifrs성장성병합>
행렬 수 :  (20947, 10)
회사 수 :  2456


### 매출액증가율

In [84]:
# k-gaap 각 기업의 2009, 2010년 재무데이터 추출(kgconsep의 마지막 행)
## 2009년(마지막-2번째)데이터
last_cap = kgconsep.groupby('회사명', as_index=False)['자산', '자본', '영업손익', '당기순이익', '매출액'].nth(-2).index
last_cap2 = kgconsep.iloc[last_cap, :]

## 2010년(마지막)데이터
last_cap3 = kgconsep.groupby('회사명', as_index=False)['자산', '자본', '영업손익', '당기순이익', '매출액'].nth(-1).index
last_cap4 = kgconsep.iloc[last_cap3, :]


# 2009, 2010 데이터 합치기
con_0910 = pd.concat([last_cap2, last_cap4])
con_0910.sort_values(by = ['회사명', '회계년도'], ignore_index=True, inplace=True)


# 2009, 2010년 데이터와 k-ifrs 합치고 날짜별 정렬
kifrs = pd.concat([con_0910, consep])
kifrs.sort_values(by = ['회사명', '회계년도'], ignore_index=True, inplace=True)


kifrs2 = kifrs.copy()

# 파생변수 만들기위한 전기 데이터 만들기
kifrs2['전기매출액'] = kifrs2['매출액'].shift().where(kifrs2.회사명.eq(kifrs2.회사명.shift()))

kifrs3 = kifrs2.copy()


# 성장성 재무비율 구하기
for i, row in kifrs3.iterrows () :
    if kifrs3.at[i,'전기매출액'] > 0 or kifrs3.at[i,'전기매출액'] < 0 and kifrs3.at[i,'매출액'] < 0:
        kifrs3.at[i,'매출액증가율_계산'] = ((kifrs3.at[i,'매출액'] - kifrs3.at[i,'전기매출액'])/abs(kifrs3.at[i,'전기매출액']))*100
    elif kifrs3.at[i,'전기매출액'] < 0 and kifrs3.at[i,'매출액'] > 0:
        kifrs3.at[i,'매출액증가율_계산'] = ((kifrs3.at[i,'매출액'] - kifrs3.at[i,'전기매출액'])/abs(kifrs3.at[i,'매출액']))*100


# 2009년 데이터는 삭제
kifrs4 = kifrs3.copy()

index_to_drop = kifrs4.groupby('회사명', as_index=False)['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익',
       '매출액', '전기매출액', '매출액증가율_계산'].nth(0).index
kifrs4.drop(kifrs4.index[index_to_drop], inplace=True)


# 2010~2020 필요한 컬럼만 남기기
kifrs5 = kifrs4[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '매출액증가율_계산']]


# 데이터 저장
kifrs5.to_csv('데이터/2차 전처리/파생변수_매출액증가율.csv', encoding='CP949', index=False)

<ipython-input-84-5ad4083320b9>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  last_cap = kgconsep.groupby('회사명', as_index=False)['자산', '자본', '영업손익', '당기순이익', '매출액'].nth(-2).index
<ipython-input-84-5ad4083320b9>:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  last_cap3 = kgconsep.groupby('회사명', as_index=False)['자산', '자본', '영업손익', '당기순이익', '매출액'].nth(-1).index
<ipython-input-84-5ad4083320b9>:40: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  index_to_drop = kifrs4.groupby('회사명', as_index=False)['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '자산', '자본', '영업손익', '당기순이익',


### 이자보상배율

In [85]:
# k-gaap 각 기업의 2010년(마지막행) 재무데이터 추출
last_cap3_ = kgaap.groupby('회사명', as_index=False)['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '통계청 한국표준산업분류 코드 10차(대분류)', '총자본증가율', '영업이익증가율',
       '순이익증가율', '자기자본증가율', '매출액증가율', '종업원수증가율', '유형자산증가율', '유동자산증가율',
       '재고자산증가율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율', '재고자산 대 순운전자본비율',
       '매출채권 대 매입채무비율', '부채비율', '이자보상배율(이자비용)', 'CASH FLOW 대 부채비율',
       'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율',
       '유동자산구성비율', '재고자산 대 유동자산비율', '타인자본구성비율', '자기자본배율(백만원)', '매출채권비율',
       '유동부채비율', '단기차입금 대 총차입금비율', '차입금의존도', '차입금비율', '종업원1인당 부가가치', '노동장비율',
       '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율',
       '매출액총이익률', '매출액영업이익률', '매출액순이익률', '총자본순이익률', '총자본영업이익률(원)',
       '자기자본영업이익률(원)', '자기자본순이익률(원)', '금융비용부담률(원)', '수지비율', '사내유보 대 자기자본비율',
       '1주당매출액', 'EPS(Earning Per Share)', 'CPS(Cash flow Per Share)',
       'BPS(Book-value Per Share)', '매출원가 대 매출액비율', '이자부담률', '지급이자율',
       '차입금평균이자율', '유보율', '총자본회전률', '자기자본회전률', '타인자본회전률', '유동자산회전률', '당좌자산회전률',
       '재고자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '유형자산회전율'].nth(-1).index
last_cap4_ = kgaap.iloc[last_cap3_, :]


k_ = last_cap4_.copy()

# 콘캣하기 위해서 컬럼 명 같게 바꾸기
k_ = k_.rename(columns={'이자보상배율(이자비용)' : '이자보상배율_계산'})
ifrscon = ifrscon.rename(columns={'이자보상배율' : '이자보상배율_계산'})


# 2010년 데이터와 k-ifrs 합치고 회계년도 순으로 정렬
k2_ = pd.concat([k_, ifrscon])
k2_.sort_values(by = ['회사명', '회계년도'], ignore_index=True, inplace=True)


# 2010~2020 이자보상배율 전처리 전 데이터 저장
k2_.to_csv('데이터/2차 전처리/파생변수_이자보상배율_전.csv', encoding='CP949', index=False)


k2_1 = k2_.copy()

# 이자보상배율이 NaN인 기업
nan_icr_list = k2_1['회사명'][k2_1['이자보상배율_계산'].isna()].index


# 이자보상배율이 NaN인 행 제거
k2_1.drop(k2_1.index[nan_icr_list], inplace=True)
k2_1.reset_index(drop=True, inplace=True)


# 이자보상배율이 999999999인 것을 해당 기업의 평균 이자보상배율 값으로 대체
k2_1.loc[k2_1['이자보상배율_계산']==999999999, '이자보상배율_계산'] = np.nan
k2_1['이자보상배율_계산'] = k2_1['이자보상배율_계산'].fillna(k2_1.groupby('회사명')['이자보상배율_계산'].transform('mean'))


# 이자보상배율 전처리 후 데이터 저장
k2_1.to_csv('데이터/2차 전처리/파생변수_이자보상배율_후.csv', encoding='CP949', index=False)


# merge
merge1_ = pd.merge(kifrs5, k2_1, on = ['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일'], how='left')


# 컬럼 순서 변경
merge1_1 = merge1_[['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '통계청 한국표준산업분류 코드 10차(대분류)', '총자본증가율', '영업이익증가율',
       '순이익증가율', '자기자본증가율', '매출액증가율', '종업원수증가율', '유형자산증가율', '유동자산증가율',
       '재고자산증가율', '자기자본구성비율', '유동비율', '당좌비율', '현금비율', '재고자산 대 순운전자본비율',
       '매출채권 대 매입채무비율', '부채비율', 'CASH FLOW 대 부채비율',
       'CASH FLOW 대 차입금비율', 'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율',
       '유동자산구성비율', '재고자산 대 유동자산비율', '타인자본구성비율', '자기자본배율(백만원)', '매출채권비율',
       '유동부채비율', '단기차입금 대 총차입금비율', '차입금의존도', '차입금비율', '종업원1인당 부가가치', '노동장비율',
       '기계장비율', '자본집약도', '총자본투자효율', '설비투자효율', '기계투자효율', '부가가치율', '노동소득분배율',
       '매출액총이익률', '매출액영업이익률', '매출액순이익률', '총자본순이익률', '총자본영업이익률(원)',
       '자기자본영업이익률(원)', '자기자본순이익률(원)', '금융비용부담률(원)', '수지비율', '사내유보 대 자기자본비율',
       '1주당매출액', 'EPS(Earning Per Share)', 'CPS(Cash flow Per Share)',
       'BPS(Book-value Per Share)', '매출원가 대 매출액비율', '이자부담률', '지급이자율',
       '차입금평균이자율', '유보율', '총자본회전률', '자기자본회전률', '타인자본회전률', '유동자산회전률', '당좌자산회전률',
       '재고자산회전률', '매출채권회전률', '순운전자본회전률', '운전자본회전률', '유형자산회전율', '업종', '정상영업이익증가율', '자기자본배율', '매출액정상영업이익률', '총자본정상영업이익률', '자기자본정상영업이익률',
       '자기자본순이익률', '금융비용부담률', '1주당순이익', '1주당 CASH FLOW', '1주당순자산', '금융손익비율',
       '매출액증가율_계산', '이자보상배율_계산']]


# 날짜 형식 변환
merge1_2 = merge1_1.copy()

merge1_2['회계년도'] = pd.to_datetime(merge1_2['회계년도'], format='%Y/%m')
merge1_2['회계년도'] = merge1_2['회계년도'].dt.strftime('%Y%m')

merge1_2['상장일'] = pd.to_datetime(merge1_2['상장일'], format='%Y/%m')
merge1_2['상장일'] = merge1_2['상장일'].dt.strftime('%Y%m')

merge1_2[['회계년도', '상장일']] = merge1_2[['회계년도', '상장일']].astype(int)


# 데이터 저장
merge1_2.to_csv('데이터/2차 전처리/파생변수_이자보상매출액.csv', encoding='CP949', index=False)


<ipython-input-85-182b38b2bd98>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  last_cap3_ = kgaap.groupby('회사명', as_index=False)['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일', '통계청 한국표준산업분류 코드 10차(대분류)', '총자본증가율', '영업이익증가율',


In [86]:
merge1_2

,회사명,거래소코드,회계년도,상장일,상장폐지일,통계청 한국표준산업분류 코드 10차(대분류),총자본증가율,영업이익증가율,순이익증가율,자기자본증가율,...,총자본정상영업이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,1주당순이익,1주당 CASH FLOW,1주당순자산,금융손익비율,매출액증가율_계산,이자보상배율_계산
0,(주)CMG제약,58820,201012,200108,NaN,21.0,5.66,0.0,0.00,82.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.861801,-7.33
1,(주)CMG제약,58820,201112,200108,NaN,NaN,1.98,NaN,0.00,-2.02,...,2.37,3.51,-8.28,0.80,-35.00,-58.60,386.51,-12.10,56.396905,3.53
2,(주)CMG제약,58820,201212,200108,NaN,NaN,109.15,NaN,0.00,75.17,...,-9.96,-16.95,-26.84,2.53,-138.00,-32.19,643.72,-7.13,-8.429134,-7.75
3,(주)CMG제약,58820,201312,200108,NaN,NaN,-2.09,NaN,0.00,55.10,...,1.40,1.96,0.74,1.65,4.16,-45.47,492.83,-1.57,25.510008,0.70
4,(주)CMG제약,58820,201412,200108,NaN,NaN,3.86,NaN,-46.64,0.01,...,0.78,0.90,0.32,-0.65,1.82,8.37,447.56,0.50,11.520576,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22748,흥아해운(주),3280,201612,197606,NaN,NaN,21.08,NaN,-263.29,11.22,...,0.67,3.23,-9.40,3.21,-170.00,-14.34,1879.15,-3.21,-1.581947,0.21
22749,흥아해운(주),3280,201712,197606,NaN,NaN,-11.43,NaN,0.00,-39.28,...,-1.45,-8.47,-47.52,3.67,-581.00,-53.97,835.26,-3.67,0.562842,-0.42
22750,흥아해운(주),3280,201812,197606,NaN,NaN,-5.33,NaN,0.00,-37.42,...,-4.55,-39.57,-91.08,4.27,-514.00,-199.43,330.21,-4.27,-9.870781,-1.16
22751,흥아해운(주),3280,201912,197606,NaN,NaN,-46.38,NaN,0.00,-57.73,...,-2.00,-23.76,-98.70,21.89,-391.00,-92.16,133.10,-16.91,-86.447605,-0.55


In [87]:
merge1_2.isna().sum()

회사명                  0
거래소코드                0
회계년도                 0
상장일                  0
상장폐지일            20898
                 ...  
1주당 CASH FLOW     3801
1주당순자산            3911
금융손익비율            3801
매출액증가율_계산         1930
이자보상배율_계산         1544
Length: 87, dtype: int64

In [88]:
merge1_2

,회사명,거래소코드,회계년도,상장일,상장폐지일,통계청 한국표준산업분류 코드 10차(대분류),총자본증가율,영업이익증가율,순이익증가율,자기자본증가율,...,총자본정상영업이익률,자기자본정상영업이익률,자기자본순이익률,금융비용부담률,1주당순이익,1주당 CASH FLOW,1주당순자산,금융손익비율,매출액증가율_계산,이자보상배율_계산
0,(주)CMG제약,58820,201012,200108,NaN,21.0,5.66,0.0,0.00,82.10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.861801,-7.33
1,(주)CMG제약,58820,201112,200108,NaN,NaN,1.98,NaN,0.00,-2.02,...,2.37,3.51,-8.28,0.80,-35.00,-58.60,386.51,-12.10,56.396905,3.53
2,(주)CMG제약,58820,201212,200108,NaN,NaN,109.15,NaN,0.00,75.17,...,-9.96,-16.95,-26.84,2.53,-138.00,-32.19,643.72,-7.13,-8.429134,-7.75
3,(주)CMG제약,58820,201312,200108,NaN,NaN,-2.09,NaN,0.00,55.10,...,1.40,1.96,0.74,1.65,4.16,-45.47,492.83,-1.57,25.510008,0.70
4,(주)CMG제약,58820,201412,200108,NaN,NaN,3.86,NaN,-46.64,0.01,...,0.78,0.90,0.32,-0.65,1.82,8.37,447.56,0.50,11.520576,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22748,흥아해운(주),3280,201612,197606,NaN,NaN,21.08,NaN,-263.29,11.22,...,0.67,3.23,-9.40,3.21,-170.00,-14.34,1879.15,-3.21,-1.581947,0.21
22749,흥아해운(주),3280,201712,197606,NaN,NaN,-11.43,NaN,0.00,-39.28,...,-1.45,-8.47,-47.52,3.67,-581.00,-53.97,835.26,-3.67,0.562842,-0.42
22750,흥아해운(주),3280,201812,197606,NaN,NaN,-5.33,NaN,0.00,-37.42,...,-4.55,-39.57,-91.08,4.27,-514.00,-199.43,330.21,-4.27,-9.870781,-1.16
22751,흥아해운(주),3280,201912,197606,NaN,NaN,-46.38,NaN,0.00,-57.73,...,-2.00,-23.76,-98.70,21.89,-391.00,-92.16,133.10,-16.91,-86.447605,-0.55


In [ ]:
label = merge1_2.copy()

label

In [73]:
df_part5 = pd.read_csv('https://raw.githubusercontent.com/jkim0313/project2_quaero/master/data/%EC%A0%84%EC%B2%98%EB%A6%AC/%EC%A0%84%EC%B2%98%EB%A6%ACpart5_final2.csv', encoding='CP949')
df_part5

,회사명,거래소코드,회계년도,상장일,상장폐지일,부실기업1,부실기업2,부실기업,업종,경기민감도,...,기계장비율_계산,자본집약도_계산,총자본회전률,자기자본회전률,타인자본회전률,유동자산회전률,당좌자산회전률,재고자산회전률,매출채권회전률,순운전자본회전률
0,(주)CMG제약,58820,201112,200108,NaN,0,0,0,제조업,1,...,19.715837,204.066667,0.66,0.984333,2.01,1.25,1.55,6.42,2.45,0.86
1,(주)CMG제약,58820,201212,200108,NaN,0,0,0,제조업,1,...,18.562172,425.291045,0.39,0.655130,0.93,0.52,0.57,5.71,1.85,0.42
2,(주)CMG제약,58820,201312,200108,NaN,0,0,0,제조업,1,...,17.604882,411.485294,0.36,0.506332,1.26,0.48,0.52,5.39,2.07,0.42
3,(주)CMG제약,58820,201412,200108,NaN,0,0,0,제조업,1,...,30.445768,354.432927,0.40,0.464339,2.85,0.68,0.80,4.77,1.86,0.56
4,(주)CMG제약,58820,201512,200108,NaN,0,0,0,제조업,1,...,28.007199,362.769231,0.47,0.564493,2.88,0.99,1.22,5.22,2.10,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16629,흥아해운(주),3280,201612,197606,NaN,0,0,0,운수 및 창고업,0,...,0.000000,1000.623142,0.95,4.964267,1.20,6.98,7.96,56.47,25.24,2.19
16630,흥아해운(주),3280,201712,197606,NaN,0,0,0,운수 및 창고업,0,...,0.000000,905.653637,0.93,4.550483,1.12,7.68,9.30,44.09,23.08,2.09
16631,흥아해운(주),3280,201812,197606,NaN,0,0,0,운수 및 창고업,0,...,0.000000,983.400754,0.91,5.407265,1.03,6.73,8.14,38.87,19.15,1.75
16632,흥아해운(주),3280,201912,197606,NaN,0,0,0,운수 및 창고업,0,...,0.000000,1140.268437,0.17,7.934063,0.18,1.01,1.16,7.70,4.78,0.28


In [74]:
# merge
merge2 = pd.merge(df_part5, merge1_2, on = ['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일'], how='left')

merge2

,회사명,거래소코드,회계년도,상장일,상장폐지일,부실기업1,부실기업2,부실기업,업종_x,경기민감도,...,총자본정상영업이익률_y,자기자본정상영업이익률_y,자기자본순이익률_y,금융비용부담률_y,1주당순이익_y,1주당 CASH FLOW_y,1주당순자산_y,금융손익비율,매출액증가율_계산,이자보상배율_계산
0,(주)CMG제약,58820,201112,200108,NaN,0,0,0,제조업,1,...,2.37,3.51,-8.28,0.80,-35.00,-58.60,386.51,-12.10,56.396905,3.53
1,(주)CMG제약,58820,201212,200108,NaN,0,0,0,제조업,1,...,-9.96,-16.95,-26.84,2.53,-138.00,-32.19,643.72,-7.13,-8.429134,-7.75
2,(주)CMG제약,58820,201312,200108,NaN,0,0,0,제조업,1,...,1.40,1.96,0.74,1.65,4.16,-45.47,492.83,-1.57,25.510008,0.70
3,(주)CMG제약,58820,201412,200108,NaN,0,0,0,제조업,1,...,0.78,0.90,0.32,-0.65,1.82,8.37,447.56,0.50,11.520576,3.60
4,(주)CMG제약,58820,201512,200108,NaN,0,0,0,제조업,1,...,-4.49,-5.37,-7.62,-0.11,-42.00,27.30,397.60,0.11,18.851969,-32.22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16629,흥아해운(주),3280,201612,197606,NaN,0,0,0,운수 및 창고업,0,...,0.67,3.23,-9.40,3.21,-170.00,-14.34,1879.15,-3.21,-1.581947,0.21
16630,흥아해운(주),3280,201712,197606,NaN,0,0,0,운수 및 창고업,0,...,-1.45,-8.47,-47.52,3.67,-581.00,-53.97,835.26,-3.67,0.562842,-0.42
16631,흥아해운(주),3280,201812,197606,NaN,0,0,0,운수 및 창고업,0,...,-4.55,-39.57,-91.08,4.27,-514.00,-199.43,330.21,-4.27,-9.870781,-1.16
16632,흥아해운(주),3280,201912,197606,NaN,0,0,0,운수 및 창고업,0,...,-2.00,-23.76,-98.70,21.89,-391.00,-92.16,133.10,-16.91,-86.447605,-0.55


In [68]:
df_part5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16634 entries, 0 to 16633
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                16634 non-null  object 
 1   거래소코드              16634 non-null  int64  
 2   회계년도               16634 non-null  int64  
 3   상장일                16634 non-null  int64  
 4   상장폐지일              747 non-null    object 
 5   부실기업1              16634 non-null  int64  
 6   부실기업2              16634 non-null  int64  
 7   부실기업               16634 non-null  int64  
 8   업종                 16634 non-null  object 
 9   경기민감도              16634 non-null  int64  
 10  총자본증가율             16634 non-null  float64
 11  순이익증가율             16634 non-null  float64
 12  자기자본증가율            16634 non-null  float64
 13  매출액증가율             16634 non-null  float64
 14  종업원수증가율            16634 non-null  float64
 15  정상영업손익증가율          16634 non-null  float64
 16  매출액정상영업이익률         166

In [227]:
# merge
merge3 = pd.merge(df_part5, merge1_3, on = ['회사명', '거래소코드', '회계년도', '상장일', '상장폐지일'], how='left')
merge3

,회사명,거래소코드,회계년도,상장일,상장폐지일,부실기업1,부실기업2,부실기업,업종_x,경기민감도,총자본증가율_x,순이익증가율_x,자기자본증가율_x,매출액증가율_x,종업원수증가율_x,정상영업손익증가율,매출액정상영업이익률_x,매출액순이익률_x,총자본순이익률_x,총자본정상영업이익률_x,자기자본정상영업이익률_x,자기자본순이익률_x,금융비용부담률_x,수지비율_x,사내유보 대 자기자본비율_x,1주당매출액_x,1주당순이익_x,1주당 CASH FLOW_x,1주당순자산_x,유보율_x,자기자본구성비율_x,유동비율_x,당좌비율_x,현금비율_x,매출채권 대 매입채무비율_x,부채비율_x,이자보상배율,CASH FLOW 대 부채비율_x,CASH FLOW 대 차입금비율_x,CASH FLOW 대 총자본비율_x,CASH FLOW 대 매출액비율_x,재고자산대순운전자본비율,총자본투자효율_x,설비투자효율_x,기계투자효율_x,부가가치율_x,노동소득분배율_x,종업원1인당부가가치율_계산,노동장비율_계산,기계장비율_계산,자본집약도_계산,총자본회전률_x,자기자본회전률_x,타인자본회전률_x,유동자산회전률_x,당좌자산회전률_x,재고자산회전률_x,매출채권회전률_x,순운전자본회전률_x,통계청 한국표준산업분류 코드 10차(대분류),총자본증가율_y,영업이익증가율,순이익증가율_y,자기자본증가율_y,매출액증가율_y,종업원수증가율_y,유형자산증가율,유동자산증가율,재고자산증가율,자기자본구성비율_y,유동비율_y,당좌비율_y,현금비율_y,재고자산 대 순운전자본비율,매출채권 대 매입채무비율_y,부채비율_y,CASH FLOW 대 부채비율_y,CASH FLOW 대 차입금비율_y,CASH FLOW 대 총자본비율_y,CASH FLOW 대 매출액비율_y,유동자산구성비율,재고자산 대 유동자산비율,타인자본구성비율,자기자본배율(백만원),매출채권비율,유동부채비율,단기차입금 대 총차입금비율,차입금의존도,차입금비율,종업원1인당 부가가치,노동장비율,기계장비율,자본집약도,총자본투자효율_y,설비투자효율_y,기계투자효율_y,부가가치율_y,노동소득분배율_y,매출액총이익률,매출액영업이익률,매출액순이익률_y,총자본순이익률_y,총자본영업이익률(원),자기자본영업이익률(원),자기자본순이익률(원),금융비용부담률(원),수지비율_y,사내유보 대 자기자본비율_y,1주당매출액_y,EPS(Earning Per Share),CPS(Cash flow Per Share),BPS(Book-value Per Share),매출원가 대 매출액비율,이자부담률,지급이자율,차입금평균이자율,유보율_y,총자본회전률_y,자기자본회전률_y,타인자본회전률_y,유동자산회전률_y,당좌자산회전률_y,재고자산회전률_y,매출채권회전률_y,순운전자본회전률_y,운전자본회전률,유형자산회전율,업종_y,정상영업이익증가율,자기자본배율,매출액정상영업이익률_y,총자본정상영업이익률_y,자기자본정상영업이익률_y,자기자본순이익률_y,금융비용부담률_y,1주당순이익_y,1주당 CASH FLOW_y,1주당순자산_y,금융손익비율,매출액증가율_계산,이자보상배율_계산
0,(주)CMG제약,58820,201112,200108,NaN,0,0,0,제조업,1,13.166482,80.280163,5.219839,56.396905,4.65,1263.950784,3.61,-8.50,-5.58,2.37,3.51,-8.28,0.80,108.35,-245.87,406.280703,-34.519834,-58.600377,418.623493,-17.45,66.046945,200.97,165.03,15.10,602.31,51.41,3.53,-46.08,-172.40,-15.65,-24.07,35.590076,12.72,35.86,1.285100e+02,19.56,89.819047,27.179630,71.503704,19.715837,204.066667,0.66,0.984333,2.01,1.25,1.55,6.42,2.45,0.86,NaN,1.98,NaN,0.00,-2.02,56.40,4.65,-5.90,17.64,-1.43,66.05,200.97,165.03,15.10,35.59,602.31,51.41,-46.08,-172.40,-15.65,-24.07,56.18,17.88,33.95,NaN,325.29,42.33,80.71,9.08,13.74,27.18,71.32,19.75,204.06,12.72,35.86,1.285100e+02,19.56,89.82,41.83,NaN,-8.50,-5.58,NaN,NaN,NaN,NaN,108.35,-245.87,406.28,NaN,NaN,NaN,58.17,1.61,7.32,8.02,-17.45,0.66,0.97,2.01,1.25,1.55,6.42,2.45,0.86,0.00,1.77,제조업,0.00,82.55,3.61,2.37,3.51,-8.28,0.80,-35.00,-58.60,386.51,-12.10,56.396905,3.53
1,(주)CMG제약,58820,201212,200108,NaN,0,0,0,제조업,1,109.147119,-341.571868,75.170505,-8.429134,-0.74,-756.819406,-25.87,-40.97,-15.78,-9.96,-16.95,-26.84,2.53,140.55,-165.71,246.198501,-100.872314,-23.817768,478.648111,-4.31,55.317408,693.49,649.98,68.81,323.41,80.77,-7.75,-8.10,-11.83,-3.62,-12.71,7.330001,0.47,2.89,1.067000e+01,1.66,1749.845902,-0.657015,69.134328,18.562172,425.291045,0.39,0.655130,0.93,0.52,0.57,5.71,1.85,0.42,NaN,109.15,NaN,0.00,75.17,-8.43,-0.74,-4.12,207.08,7.73,55.32,693.49,649.98,68.81,7.33,323.41,80.77,-8.10,-11.83,-3.62,-12.71,82.49,6.27,44.68,NaN,292.29,21.50,0.57,30.58,55.28,-0.66,69.14,18.64,425.29,0.47,2.89,1.067000e+01,1.66,1749.87,28.34,NaN,-40.97,-15.78,NaN,NaN,NaN,NaN,140.55,-165.71,332.74,NaN,NaN,NaN,71.66,2.37,3.11,5.45,-4.31,0.39,0.66,0.93,0.52,0.57,5.71,1.85,0.42,0.00,1.71,제조업,0.00,95.69,-25.87,-9.96,-16.95,-26.84,2.53,-138.00,-32.19,643.72,-7.13,-8.429134,-7.75
2,(주)CMG제약,58820,201312,200108,NaN,0,0,0,제조업,1,-2.093352,2348.675303,55.100407,25.510008,1.49,631.501169,3.88,1.45,0.52,1.40,1.96,0.74,1.65,98.55,-106.33,235.517466,3.419043,-45.471032,566.198482,13.12,87.631971,694.71,610.74,394.63,282.72,14.11,0.70,-39.85,-194.73,-4.93,-13.51,14.120364,12.51,77.91,2.894000e+02,34.29,71.284323,51.118235,79.448529,17.604882,411.485294,0.36,0.506332,1.26,0.48,0.52,5.39,2.07,0.42,NaN,-2.09,NaN,0.00,55.10,25.51,1.49,16.09,-18.89,56.29,87.63,694.71,610.74,394.63,14.12,282.72,14.11,-39.85,-194.73,-4.93,-13.51,68.34,12.09,12.37,NaN,239.46,11.23,0.00,2.53,2.89,51.12,66.01,17.77,411.48,12.51,77.91,2.894000e+02,34.29,71.28,38.70,NaN,1.45,0.52,NaN,NaN,NaN,NaN,98.55,-106.33,235.52,NaN,NaN,NaN

In [212]:
merge1_2 = merge1_1.copy()

In [213]:
merge1_2 = merge1_1.copy()

# 전처리part4과 합치기 위해서 같은 형태로 변환
merge1_2['회계년도'] = pd.to_datetime(merge1_2['회계년도'], format='%Y/%m')
merge1_2['회계년도'] = merge1_2['회계년도'].dt.strftime('%Y%m')

merge1_2['상장일'] = pd.to_datetime(merge1_2['상장일'], format='%Y/%m')
merge1_2['상장일'] = merge1_2['상장일'].dt.strftime('%Y%m')

merge1_2[['회계년도', '상장일']] = merge1_2[['회계년도', '상장일']].astype(int)